#### Pipeline ETL para ingesta en base de datos relacional

In [8]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import gzip

In [9]:
# Cambiar el directorio de trabajo al directorio donde se encuentran los archivos JSON
%cd ./metadata
# Obtener una lista de archivos en el directorio actual
archivos = os.listdir()

# Crear un DataFrame vacío para almacenar los datos
df_metadata = pd.DataFrame()

# Iterar sobre cada archivo en la lista de archivos
for archivo in archivos:
    # Verificar si el archivo tiene la extensión .json.gz
    if archivo.endswith('.json.gz'):
        # Leer el archivo JSON comprimido y cargarlo en un DataFrame
        data = pd.read_json(archivo, lines=True, compression='gzip')
        # Concatenar los datos del archivo al DataFrame principal
        df_metadata = pd.concat([df_metadata, data], axis=0)
        
print("finalizo cargan de datos")


c:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\pipeline ETL\google\metadata
finalizo cargan de datos


In [10]:
df_metadata.head(2)

,0,1,2,3,4,5,6,7,8,9,...,274991,274992,274993,274994,274995,274996,274997,274998,274999,275000
0,"{'name': 'Porter Pharmacy', 'address': 'Porter...","{'name': 'City Textile', 'address': 'City Text...","{'name': 'San Soo Dang', 'address': 'San Soo D...","{'name': 'Nova Fabrics', 'address': 'Nova Fabr...","{'name': 'Nobel Textile Co', 'address': 'Nobel...","{'name': 'Matrix International Textiles', 'add...","{'name': 'Vons Chicken', 'address': 'Vons Chic...","{'name': 'Polished Nail Salon & Spa', 'address...","{'name': 'Sweet Rewards Gluten Free Bakery, LL...","{'name': 'PeepNotes', 'address': None, 'gmap_i...",...,"{'name': 'Southside Bank', 'address': 'Southsi...","{'name': 'Smart Services', 'address': 'Smart S...","{'name': 'St John Baptist Church', 'address': ...","{'name': 'New South Services', 'address': None...","{'name': 'Stone Castle Fence', 'address': None...","{'name': 'Maven Realty', 'address': 'Maven Rea...","{'name': 'Stone Castle Fence', 'address': 'Sto...","{'name': 'Health Insurance Solutions Team', 'a...","{'name': 'Bernard Building Center', 'address':...","{'name': 'HMR Motor Works and Auto Care', 'add..."
0,"{'name': 'Porter Pharmacy', 'address': 'Porter...","{'name': 'Glik's', 'address': 'Glik's, 1405 W ...",{'name': 'Oscoda County Area Transit Authority...,"{'name': 'Hickerson Floor & Tile Haus', 'addre...","{'name': 'Beck Suzanne A MD', 'address': 'Beck...","{'name': 'Perfection Plus Quality', 'address':...","{'name': 'Charlie's Hair Salon', 'address': 'C...","{'name': 'Double Header', 'address': 'Double H...","{'name': 'AACCT', 'address': 'AACCT, 34-48 Lin...","{'name': 'Glen Head Racquet Club', 'address': ...",...,"{'name': 'Bedford Auto Service', 'address': 'B...","{'name': 'Sandbox Bar & Grill', 'address': 'Sa...","{'name': 'Clark Street Brentwood', 'address': ...","{'name': 'Up In Smoke', 'address': 'Up In Smok...","{'name': 'Slabaugh Says', 'address': 'Slabaugh...","{'name': 'Quality Office Supply', 'address': '...","{'name': 'JV towing', 'address': 'JV towing, 4...","{'name': 'Sisom Osia, MD (Gateway Medical Cent...","{'name': 'Rasa Floors', 'address': 'Rasa Floor...","{'name': 'Alpha Testing, Inc.', 'address': 'Al..."


In [11]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description', 'price'], inplace=True)

KeyError: "['address', 'state', 'relative_results', 'url', 'hours', 'description', 'price'] not found in axis"

In [ ]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [ ]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "miselaneos"
gmap_id = df_metadata.copy()

# Resetear el índice del DataFrame df_gmap_id
gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
miselaneos = pd.concat([gmap_id, df_desanidado], axis=1)

In [ ]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [ ]:
# Creamos funcion para quedarnos con la primera categoria en la columna category
def primer_categoria(lista):
    if lista and len(lista) >= 1:
        return lista[0]
    else:
        return None  

# Aplicar la función a cada fila de la columna 'category'
df_metadata['categoria'] = df_metadata['category'].apply(primer_categoria)

In [ ]:
# Eliminamos columna category
df_metadata.drop(columns=['category'], inplace=True)

In [ ]:
# Volvemos a nombrar la columna categoria por category por coherencia linguistica
df_metadata.rename(columns={'categoria': 'category'}, inplace=True)

In [ ]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

False    2173428
True       26580
Name: count, dtype: int64

In [ ]:
# Eliminamos duplicados
df_metadata.drop_duplicates(inplace=True)

In [ ]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()

name                 31
gmap_id               0
latitude              0
longitude             0
avg_rating            0
num_of_reviews        0
categoria         13945
dtype: int64

In [ ]:
# Eliminar filas con valores nulos en las columnas 'categoria_1' y 'name'
df_metadata = df_metadata.dropna(subset=['categoria', 'name'])

# Verificamos nuevamente
df_metadata.isnull().sum()


name              0
gmap_id           0
latitude          0
longitude         0
avg_rating        0
num_of_reviews    0
categoria         0
dtype: int64

In [ ]:
# cambio el nombre de la columna gmap_id a business_id 
df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)

C:\Users\sebas\AppData\Local\Temp\ipykernel_27776\1262992948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata.rename(columns={'gmap_id': 'business_id'}, inplace=True)


In [ ]:
# Verificamos tamaño de dataset
df_metadata.shape

In [ ]:
# Observamos resultados
df_metadata.head(2)

,name,business_id,latitude,longitude,avg_rating,num_of_reviews,categoria
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.35710,4.9,16,Pharmacy
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.21529,4.5,6,Textile exporter


In [ ]:
# Exportamos el dataset
df_metadata.to_csv('metadata.csv', index=False)
df_metadata.to_parquet('metadata.parquet')